In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F

In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
from torchvision.datasets import ImageFolder
from PIL import Image

# Criar um objeto ImageFolder
dataset = ImageFolder(r"C:\Users\paulo\Área de Trabalho\codigos\python\Untitled Folder\redes_neurais\Dogs\train")

# Obter o caminho para uma imagem específica no dataset
image_path = dataset.imgs[0][0]  # Exemplo: primeira imagem do dataset

# Abrir a imagem usando PIL
image = Image.open(image_path)

# Obter o tamanho da imagem
image_size = image.size
print(image_size)
#necessário resize

(224, 224)


In [6]:
mean = [0.485, 0.456, 0.406]  # Médias dos canais R, G e B nas imagens do conjunto de treinamento
std = [0.229, 0.224, 0.225]  # Desvios padrão dos canais R, G e B nas imagens do conjunto de treinamento

transforms_treino_1 = torchvision.transforms.Compose([
       torchvision.transforms.ToTensor(),
       torchvision.transforms.Normalize(mean=mean, std=std)  # Normaliza os valores dos canais de cor
])



transforms_treino_2 = torchvision.transforms.Compose([
       torchvision.transforms.ColorJitter(brightness=0.3, contrast=0.25, saturation=0.4, hue=0.12),
       torchvision.transforms.GaussianBlur(kernel_size=(7, 13), sigma=(0.1, 0.2)),
       torchvision.transforms.ToTensor(),
       torchvision.transforms.Normalize(mean=mean, std=std)  # Normaliza os valores dos canais de cor
])

transforms_treino_3 = torchvision.transforms.Compose([
       torchvision.transforms.RandomHorizontalFlip(),
       torchvision.transforms.RandomRotation(30),
       torchvision.transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
       torchvision.transforms.ToTensor(),
       torchvision.transforms.Normalize(mean=mean, std=std)  # Normaliza os valores dos canais de cor
    ])





transforms_validacao = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=mean, std=std)
])

transforms_teste = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=mean, std=std)
])

#batches
batch_size = 48



trainset_1 = torchvision.datasets.ImageFolder(r"C:\Users\paulo\Área de Trabalho\codigos\python\Untitled Folder\redes_neurais\Dogs\train", transform = transforms_treino_1)
trainset_2 = torchvision.datasets.ImageFolder(r"C:\Users\paulo\Área de Trabalho\codigos\python\Untitled Folder\redes_neurais\Dogs\train", transform = transforms_treino_2)
trainset_3 = torchvision.datasets.ImageFolder(r"C:\Users\paulo\Área de Trabalho\codigos\python\Untitled Folder\redes_neurais\Dogs\train", transform = transforms_treino_3)


validset = torchvision.datasets.ImageFolder(r"C:\Users\paulo\Área de Trabalho\codigos\python\Untitled Folder\redes_neurais\Dogs\valid", transform = transforms_validacao)
testset = torchvision.datasets.ImageFolder(r"C:\Users\paulo\Área de Trabalho\codigos\python\Untitled Folder\redes_neurais\Dogs\test", transform = transforms_teste)




# loaders for data
trainloader_1 = torch.utils.data.DataLoader(trainset_1 , batch_size=batch_size , shuffle = True)
trainloader_2 = torch.utils.data.DataLoader(trainset_2 , batch_size=batch_size , shuffle = True)
trainloader_3 = torch.utils.data.DataLoader(trainset_3 , batch_size=batch_size , shuffle = True)


validloader = torch.utils.data.DataLoader(validset , batch_size=batch_size , shuffle = False)
testloader = torch.utils.data.DataLoader(testset  , batch_size=batch_size)

In [7]:
in_channels, growth_rate = 64, 32
tamanho_bottleneck = [3,5,7,4]
camada_intermediaria = []



## Camada inicial ##
## (BN - Relu - Conv)  + Max pool ##

camada_inicial = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(7,7), stride=2, padding=3), # convolucional
    nn.BatchNorm2d(64), #batch normalization
    nn.ReLU(), # relu ativação
    nn.MaxPool2d(kernel_size=(3,3), stride=2, padding=1)) # Max pool

## Bottleneck Block (mini bloco) ~ compõe o bloco denso ##
## (BN - Relu - Conv) + dropout + (BN - Relu - Conv) + dropout ##

def bottleneck(in_channels, out_channels, dropout=0.2):
    k = nn.Sequential(
        nn.BatchNorm2d(in_channels),
        nn.ReLU(),
        nn.Conv2d(in_channels, out_channels, kernel_size=(1,1), padding=0, stride=1),
        nn.Dropout(p=dropout),

        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=(3,3), padding=1, stride=1),
        nn.Dropout(p=dropout))
    return k


## Dense Block (Bloco denso) ##
## União de várias Bottleneck Block (mini blocos) ##

class DenseBlock(nn.Module):
    def __init__(self, tamanho, in_channels, growth_rate):
        super().__init__()
        self.net = nn.Sequential()
        for i in range(tamanho):
            self.net.add_module(f'bottleneck_{i+1}', bottleneck(in_channels + i * growth_rate, growth_rate))

    def forward(self, X):
        for SS in self.net:
            Y = SS(X)
            X = torch.cat((X, Y), dim=1)
        return X





## Transition Block (Bloco de transição) ##
## (BN - Relu - Conv) + dropout + Average Pooling

class Transition(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.2):
        super().__init__()

        self.batch_norm = nn.BatchNorm2d(in_channels)

        self.relu = nn.ReLU(inplace=True)

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1,
                               padding=0, bias=False)

        self.dropout = nn.Dropout(p=dropout)

        self.avgpool = nn.AvgPool2d(kernel_size=(2,2), stride = 2)

    def forward(self, x):
        saida = self.conv1(self.relu(self.batch_norm(x)))

        saida = self.dropout(saida)

        saida = self.avgpool(saida)

        return saida

## Camada intermediária ##
## Junção de dense block + transition + dense block ##


for i, tamanho in enumerate(tamanho_bottleneck):
    camada_intermediaria.append(DenseBlock(tamanho, in_channels, growth_rate))

    in_channels += tamanho * growth_rate

    if i != len(tamanho_bottleneck) - 1:
        camada_intermediaria.append(Transition(in_channels, in_channels // 2))
        in_channels = in_channels // 2


## Classification Layer (Camada de classificação) ##
## BN + Relu + Global AVG Pool + Flatten (diminuir a dimensão) + fully connected ##
camada_final = nn.Sequential(
    nn.BatchNorm2d(in_channels),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(in_channels, 70))


## DenseNet ##
## Junção de tomas as camadas anteriores ##

DenseNet = nn.Sequential(
    camada_inicial,
    *camada_intermediaria,
    camada_final)

## Treino
loss_fn = nn.CrossEntropyLoss()

def train(model, optimizer, train_loader):
    model.train()

    batch_loss = 0.0

    for inputs, labels in train_loader:
        # joga pra GPU, meu filho
        inputs, labels = inputs.to(device, non_blocking = True), labels.to(device, non_blocking = True)

        #forward
        y_hat = model(inputs)
        loss = loss_fn(y_hat, labels)

        optimizer.zero_grad()

        loss.backward() #backward
        optimizer.step() #weight updates

        batch_loss += float(loss)

    train_loss = batch_loss / len(train_loader)

    return train_loss

## teste 

def test(model, optimizer, test_loader):
    model.eval()

    batch_correct = 0
    batch_loss = 0.0

    for inputs, labels in test_loader:
        model.eval()
        inputs, labels = inputs.to(device, non_blocking = True), labels.to(device, non_blocking = True)

        #nao calcular os gradientes (estamos testando o modelo)
        with torch.no_grad():
            y_hat = model(inputs)
            loss = loss_fn(y_hat, labels)

        batch_loss += float(loss)
        batch_correct += (torch.argmax(y_hat,1) == labels).float().sum()

    test_loss = batch_loss/len(test_loader)
    test_acc = batch_correct/len(test_loader.dataset)

    return test_loss, test_acc

#funcao que executa tudo

def train_n_epochs(model, optimizer, train_loader, val_loader, test_loader, n_epochs, nome):
    hist_loss_train = []
    hist_loss_validation = []
    best_epoca,best_acc = 0,0
    score_1 = 0

    for epoch in range(n_epochs):
        train_loss = train(model, optimizer, train_loader)
        val_loss, val_acc = test(model, optimizer, val_loader)
        test_loss, test_acc = test(model,optimizer, test_loader)
        
        score = float(val_acc/val_loss)
        
        if score > score_1:
                save_model(model,nome)
                score_1 = score

        hist_loss_train.append(train_loss)
        hist_loss_validation.append(val_loss)

        print(f"Epoch {epoch}: Train Loss: {train_loss:.2f} Validation Loss: {val_loss:.2f} Validation Accuracy: {val_acc:.2f} Test Accuracy: {test_acc:.2f}")
                                   
        if test_acc > best_acc:
              best_epoca = epoch
              best_acc = test_acc
                                   
                                   
    return hist_loss_train, hist_loss_validation,best_epoca,best_acc

def save_model(model, nome):
    torch.save(model.state_dict(), nome)



In [8]:
model = DenseNet
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
hist_loss_train_1, hist_loss_validation_1,best_epoca_1,best_acc_1 = train_n_epochs(model, optimizer, trainloader_1, validloader, testloader, 120, 'modelo_1_x.pt')

Epoch 0: Train Loss: 3.87 Validation Loss: 3.74 Validation Accuracy: 0.07 Test Accuracy: 0.09
Epoch 1: Train Loss: 3.49 Validation Loss: 3.59 Validation Accuracy: 0.14 Test Accuracy: 0.16
Epoch 2: Train Loss: 3.22 Validation Loss: 3.47 Validation Accuracy: 0.15 Test Accuracy: 0.16
Epoch 3: Train Loss: 2.99 Validation Loss: 3.36 Validation Accuracy: 0.16 Test Accuracy: 0.19
Epoch 4: Train Loss: 2.81 Validation Loss: 3.29 Validation Accuracy: 0.18 Test Accuracy: 0.20
Epoch 5: Train Loss: 2.65 Validation Loss: 2.68 Validation Accuracy: 0.30 Test Accuracy: 0.30
Epoch 6: Train Loss: 2.47 Validation Loss: 3.00 Validation Accuracy: 0.27 Test Accuracy: 0.29
Epoch 7: Train Loss: 2.32 Validation Loss: 3.75 Validation Accuracy: 0.21 Test Accuracy: 0.23
Epoch 8: Train Loss: 2.17 Validation Loss: 2.75 Validation Accuracy: 0.32 Test Accuracy: 0.39
Epoch 9: Train Loss: 2.04 Validation Loss: 2.88 Validation Accuracy: 0.33 Test Accuracy: 0.34
Epoch 10: Train Loss: 1.90 Validation Loss: 2.28 Validation 

Epoch 87: Train Loss: 0.16 Validation Loss: 2.13 Validation Accuracy: 0.74 Test Accuracy: 0.75
Epoch 88: Train Loss: 0.15 Validation Loss: 2.08 Validation Accuracy: 0.73 Test Accuracy: 0.75
Epoch 89: Train Loss: 0.16 Validation Loss: 2.35 Validation Accuracy: 0.71 Test Accuracy: 0.72
Epoch 90: Train Loss: 0.16 Validation Loss: 2.49 Validation Accuracy: 0.69 Test Accuracy: 0.73
Epoch 91: Train Loss: 0.16 Validation Loss: 2.17 Validation Accuracy: 0.74 Test Accuracy: 0.77
Epoch 92: Train Loss: 0.14 Validation Loss: 2.15 Validation Accuracy: 0.73 Test Accuracy: 0.75
Epoch 93: Train Loss: 0.16 Validation Loss: 2.24 Validation Accuracy: 0.72 Test Accuracy: 0.73
Epoch 94: Train Loss: 0.15 Validation Loss: 2.50 Validation Accuracy: 0.69 Test Accuracy: 0.70
Epoch 95: Train Loss: 0.16 Validation Loss: 2.35 Validation Accuracy: 0.70 Test Accuracy: 0.76
Epoch 96: Train Loss: 0.14 Validation Loss: 2.25 Validation Accuracy: 0.74 Test Accuracy: 0.72
Epoch 97: Train Loss: 0.16 Validation Loss: 2.27 V

In [9]:
model.load_state_dict(torch.load('modelo_1_x.pt'))
model = model.to(device)

hist_loss_train_2, hist_loss_validation_2,best_epoca_2,best_acc_2 = train_n_epochs(model, optimizer, trainloader_2, validloader, testloader, 120, 'modelo_2_x.pt')

Epoch 0: Train Loss: 1.19 Validation Loss: 1.90 Validation Accuracy: 0.64 Test Accuracy: 0.65
Epoch 1: Train Loss: 0.88 Validation Loss: 1.87 Validation Accuracy: 0.66 Test Accuracy: 0.68
Epoch 2: Train Loss: 0.74 Validation Loss: 1.78 Validation Accuracy: 0.66 Test Accuracy: 0.68
Epoch 3: Train Loss: 0.68 Validation Loss: 2.73 Validation Accuracy: 0.56 Test Accuracy: 0.58
Epoch 4: Train Loss: 0.63 Validation Loss: 1.83 Validation Accuracy: 0.70 Test Accuracy: 0.70
Epoch 5: Train Loss: 0.59 Validation Loss: 1.83 Validation Accuracy: 0.68 Test Accuracy: 0.71
Epoch 6: Train Loss: 0.56 Validation Loss: 1.69 Validation Accuracy: 0.70 Test Accuracy: 0.73
Epoch 7: Train Loss: 0.54 Validation Loss: 1.81 Validation Accuracy: 0.69 Test Accuracy: 0.72
Epoch 8: Train Loss: 0.54 Validation Loss: 1.65 Validation Accuracy: 0.73 Test Accuracy: 0.77
Epoch 9: Train Loss: 0.51 Validation Loss: 1.93 Validation Accuracy: 0.68 Test Accuracy: 0.70
Epoch 10: Train Loss: 0.49 Validation Loss: 1.78 Validation 

Epoch 87: Train Loss: 0.16 Validation Loss: 2.14 Validation Accuracy: 0.76 Test Accuracy: 0.79
Epoch 88: Train Loss: 0.15 Validation Loss: 2.07 Validation Accuracy: 0.76 Test Accuracy: 0.80
Epoch 89: Train Loss: 0.16 Validation Loss: 1.95 Validation Accuracy: 0.78 Test Accuracy: 0.80
Epoch 90: Train Loss: 0.14 Validation Loss: 2.10 Validation Accuracy: 0.77 Test Accuracy: 0.76
Epoch 91: Train Loss: 0.15 Validation Loss: 2.09 Validation Accuracy: 0.74 Test Accuracy: 0.77
Epoch 92: Train Loss: 0.15 Validation Loss: 2.03 Validation Accuracy: 0.77 Test Accuracy: 0.79
Epoch 93: Train Loss: 0.14 Validation Loss: 1.67 Validation Accuracy: 0.80 Test Accuracy: 0.81
Epoch 94: Train Loss: 0.15 Validation Loss: 2.25 Validation Accuracy: 0.75 Test Accuracy: 0.79
Epoch 95: Train Loss: 0.14 Validation Loss: 2.08 Validation Accuracy: 0.77 Test Accuracy: 0.78
Epoch 96: Train Loss: 0.14 Validation Loss: 2.35 Validation Accuracy: 0.76 Test Accuracy: 0.77
Epoch 97: Train Loss: 0.14 Validation Loss: 2.04 V

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
model = DenseNet
model = model.to(device)
model.load_state_dict(torch.load('modelo_2_x.pt'))
model = model.to(device)
class_names = testset.classes

def evaluate_model(model, dataloader, device, class_names):
    model.eval()
    predictions = []
    targets = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)
            _, preds = torch.max(probabilities, dim=1)

            predictions.extend(preds.cpu().tolist())
            targets.extend(labels.cpu().tolist())

    accuracy = accuracy_score(targets, predictions)
    confusion = confusion_matrix(targets, predictions)
    report = classification_report(targets, predictions, target_names=class_names)
    
    return accuracy, confusion, report


# Supondo que você tenha a lista 'class_names' com os nomes das classes na ordem correta

accuracy, confusion, report = evaluate_model(model, testloader, device, class_names)

print(report)

                   precision    recall  f1-score   support

           Afghan       0.75      0.90      0.82        10
 African Wild Dog       1.00      1.00      1.00        10
         Airedale       0.80      0.80      0.80        10
American Hairless       1.00      0.60      0.75        10
 American Spaniel       0.64      0.70      0.67        10
          Basenji       0.56      0.90      0.69        10
           Basset       0.75      0.90      0.82        10
           Beagle       1.00      0.90      0.95        10
   Bearded Collie       1.00      0.70      0.82        10
         Bermaise       0.91      1.00      0.95        10
     Bichon Frise       1.00      0.90      0.95        10
         Blenheim       0.69      0.90      0.78        10
       Bloodhound       1.00      1.00      1.00        10
         Bluetick       0.89      0.80      0.84        10
    Border Collie       1.00      0.40      0.57        10
           Borzoi       0.90      0.90      0.90       